## ПолныйПоиск

In [1]:
def auth(): # Авторизация + скачивание данных
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)
    
    file = drive.ListFile({'q': "title = 'Nice2MeetYou' "}).GetList()[0] # Онлайн кабинет
    file.GetContentFile("Ввод/" + file['title'] + '.xlsx', mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    global fileNice2
    fileNice2 = "Ввод/" + file['title'] + '.xlsx'    
    print('Файл скачан: {}.'.format(file['title'] + '.xlsx'))
     
    url = 'https://10education.ru/CompleteSearch/select.php'
    r = requests.get(url)
    first_list = r.text.split('<br />')
    index_pass = first_list[0].index('/html')
    first_list[0] = first_list[0][index_pass+10:]
    second_list = [string.split('|') for string in first_list]
    df = pd.DataFrame(second_list)
    df = df[[0, 4]]
    df = df.rename(columns={0: 'Имя', 4: 'Первичный П'})
    global filePrimaryP
    filePrimaryP = df
    print('Первичный Процент, всего {} строк'.format(len(df)))    
    
    folder_root_id = drive.ListFile({'q': "title = 'НовоеТГ' "}).GetList()[0]['id'] # Telegram каналы
    global filesTelegram
    filesTelegram = []
    for file in drive.ListFile({'q': "title contains '.json' and trashed = false and '%s' in parents" % (folder_root_id)}).GetList():
        file.GetContentFile("Ввод/" + file['title'])
        filesTelegram.append("Ввод/" + file['title'])
        print('Файл скачан: {}.'.format(file['title']))
    print('Telegram: '+str(len(filesTelegram)))
        
    folder_root_id = drive.ListFile({'q': "title = 'НовоеИгры' "}).GetList()[0]['id'] # Построй компанию / продай компанию
    global filesPKPK
    filesPKPK = []
    for file in drive.ListFile({'q': "title contains '.html' and trashed = false and '%s' in parents" % (folder_root_id)}).GetList():
        file.GetContentFile("Ввод/" + file['title'])
        filesPKPK.append("Ввод/" + file['title'])
        print('Файл скачан: {}.'.format(file['title']))
    print('PKPK: '+str(len(filesPKPK)))

In [2]:
def get_names(df): # Все имена из таблицы
    try:
        names = df[(df['type'] == 'service') & (df['action'] == 'invite_members')].explode('members')['members'].unique()
        names = [name for name in names if name and re.match("^[a-zA-Zа-яёА-ЯЁ0-9_. ]*$", name)]
    except:
        names = [x for x in list(set(df['actor'])) if str(x) != 'nan']
    return names

In [3]:
def get_messages(df, name): # Все сообщения Telegram
    messages = df[(df['type'] == 'message') & (df['from'] == name)]['text']
    return messages

In [4]:
def transliterate(string): # Транслитерация
    let_dict = {u'а': u'a', u'б': u'b', u'в': u'v', u'г': u'g', u'д': u'd', u'е': u'e', u'ё': u'e', u'ж': u'zh', u'з': u'z',
                    u'и': u'i', u'й': u'y', u'к': u'k', u'л': u'l', u'м': u'm', u'н': u'n', u'о': u'o', u'п': u'p', u'р': u'r',
                    u'с': u's', u'т': u't', u'у': u'u', u'ф': u'f', u'х': u'h', u'ц': u'ts', u'ч': u'ch', u'ш': u'sh',
                    u'щ': u'sch', u'ъ': u'', u'ы': u'y', u'ь': u'', u'э': u'e', u'ю': u'yu', u'я': u'ya',} 
    result = u""
    for index, char in enumerate(string.lower()):
        if char in let_dict:
            char = let_dict[char]
        result += char
    return result.title()

In [5]:
def in_df(df, name): # Имя уже есть
    result = False
    if name in df.index.to_list():
        result = name
    else:
        for name1 in df.index.to_list():
            if ' ' in name and all([transliterate(name_part) in transliterate(name1) for name_part in name.split(' ')]):
                result = name1
                break
    return(result)

In [6]:
def date_format(string): # Форматирование даты
    ymd = string.split(' ')[0].split('-')
    
    return ymd[2] + '.' + ymd[1] + '.' + ymd[0]

In [7]:
def telegram_count(files, name): # Счётчик каналов и сообщений
    d_count = []
    c_count = 0
    m_count = 0
    for file, df in files.items():
        if name in get_names(df):
            d_count.append(file[5:-5])
            c_count += 1
            m_count += len(get_messages(df, name).to_list())
    return (d_count, c_count, m_count)

In [8]:
def pkpk_count(files, name): # Счётчик игр и баланса
    d_count = []
    g_count = 0
    b_count = []
    p_count = []
    for file, df in files.items():
        if name in df['Имя'].to_list():
            d_count.append(file[5:file.rfind(' ')])
            g_count += 1
            b_count.append(int(df[df['Имя'] == name]['Баланс'].values[0]))
            p_count.append(1 if (df[df['Имя'] == name]['Роль'] == 'Предприниматель').values[0] else 0)
    return (d_count, g_count, b_count, p_count)

In [9]:
def get_names_complete(file0, file1, Tfiles, Pfiles, time): # Все имена
    df0 = pd.read_excel(fileNice2, dtype = str, keep_default_na = False)
    df0['ma_email'] = df0['ma_email'].str.lower()
    df0['ma_name'] = df0['ma_email'].map(dict(df0.groupby('ma_email')['name'].apply(lambda x: max(x, key=len))))
    df0 = df0[df0['sent'].astype('datetime64[ns]') > time]
    df0.drop_duplicates(subset=['name'], keep='last', inplace=True)
    df0.drop_duplicates(subset=['ma_email'], keep='last', inplace=True)
    Nnames = df0['ma_name'].to_list()
    df1 = filePrimaryP
    df0['Процент Больший'] = [df1[df1['Имя'] == name]['Первичный П'].values[0] if name in df1['Имя'].values else '0' for name in Nnames]

    Tnames = []
    Tfiles_dict = {}
    for file in Tfiles:
        data = pd.read_json(file, encoding = 'utf-8')
        df1 = pd.DataFrame.from_records(data['messages'])
        Tnames += list(get_names(df1))
        Tnames = list(dict.fromkeys(Tnames))
        Tfiles_dict[file] = df1

    Pnames = []
    Pfiles_dict = {}
    for file in Pfiles:
        data_list = pd.read_html(file,encoding = 'utf-8')
        df2 = max([data for data in data_list if all(col in data.columns for col in ['Имя', 'Роль'])], key=len)
        Pnames += df2['Имя'].to_list()
        Pnames = list(dict.fromkeys(Pnames))
        Pfiles_dict[file] = df2
    
    global df # Таблица всех имен
    df = pd.DataFrame({'Имя': Nnames})
    df['Город проживания'] = df0['gorodprojivaniya'].to_list()
    df['Онлайн-кабинет'] = 1
    df['Дата о-к'] = df0['sent'].apply(date_format).to_list()
    df['Первичный П'] = df0['Процент Больший'].to_list()
    df['Telegram каналов'] = 0
    df['Дата Т'] = 'нет'
    df['Сообщений Т'] = 0
    df['Игры ПКПК'] = 0
    df['Роль П'] = 'нет'
    df['Дата П'] = 'нет'
    df['Баланс П'] = 'нет'
    df.set_index('Имя', inplace=True)
    print(len(df))
    
    for name in Tnames:
        d_count, c_count, m_count = telegram_count(Tfiles_dict, name)
        name_new = in_df(df, name)
        if name_new:
            if df.at[name_new, 'Дата Т'] == 'нет':
                df.at[name_new, 'Дата Т'] = d_count
            else:
                df.at[name_new, 'Дата Т'] += d_count
            df.at[name_new, 'Telegram каналов'] += c_count
            df.at[name_new, 'Сообщений Т'] += m_count
        else:
            row_default = {'Город проживания': 'не указан', 'Онлайн-кабинет': 0, 'Дата о-к': 'нет', 'Первичный П': 0, 'Telegram каналов': 0, 'Дата Т': 'нет', 'Сообщений Т': 0, 'Игры ПКПК': 0, 'Роль П': 'нет', 'Дата П': 'нет', 'Баланс П': 'нет'}
            row_default.update({'Дата Т': d_count, 'Telegram каналов': c_count, 'Сообщений Т': m_count})
            df.loc[name] = row_default
    print(len(df))
   
    for name in Pnames:
        d_count, g_count, b_count, p_count = pkpk_count(Pfiles_dict, name)
        name_new = in_df(df, name)
        if name_new:
            if df.at[name_new, 'Дата П'] == 'нет':
                df.at[name_new, 'Дата П'] = d_count
                df.at[name_new, 'Баланс П'] = b_count
                df.at[name_new, 'Роль П'] = p_count
            else:
                df.at[name_new, 'Дата П'] += d_count
                df.at[name_new, 'Баланс П'] += b_count
                df.at[name_new, 'Роль П'] += p_count
            df.at[name_new, 'Игры ПКПК'] += g_count
        else:
            row_default = {'Город проживания': 'не указан', 'Онлайн-кабинет': 0, 'Дата о-к': 'нет', 'Первичный П': 0, 'Telegram каналов': 0, 'Дата Т': 'нет', 'Сообщений Т': 0, 'Игры ПКПК': 0, 'Роль П': 'нет', 'Дата П': 'нет', 'Баланс П': 'нет'}
            row_default.update({'Дата П': d_count, 'Игры ПКПК': g_count, 'Роль П': p_count, 'Баланс П': b_count})
            df.loc[name] = row_default
    print(len(df))
            
    return(df)

In [11]:
def select_df(url):
    r = requests.get(url)
    first_list = r.text.split('<br />')
    index_pass = first_list[0].index('/html')
    first_list[0] = first_list[0][index_pass+10:]
    second_list = [string.split('|') for string in first_list]
    df = pd.DataFrame(second_list)
    columns_names = {0: 'Имя', 1: 'Город проживания', 2: 'Онлайн-кабинет', 3: 'Дата о-к', 4: 'Первичный П', 5: 'Telegram каналов', 6: 'Дата Т',
                     7: 'Сообщений Т', 8: 'Игры ПКПК', 9: 'Роль П', 10: 'Дата П', 11: 'Баланс П'}
    df = df.rename(columns=columns_names)
    df = df[:-1]
    for col in ['Telegram каналов', 'Сообщений Т', 'Игры ПКПК']:
        df[col] = df[col].astype(int)
    for col in ['Дата Т', 'Роль П', 'Дата П', 'Баланс П']:
        df[col] = df[col].apply(lambda x: x.split(', ') if x != 'нет' else x)
    return(df)

In [13]:
def update_row(last, test):
    if test['Дата Т'] == 'нет':
        pass
    else:
        if last['Дата Т'] == 'нет':
            last['Telegram каналов'] = test['Telegram каналов']
            last['Дата Т'] = test['Дата Т']
            last['Сообщений Т'] = test['Сообщений Т']
        else:
            for t in test['Дата Т']:
                if t not in last['Дата Т']:
                    last['Telegram каналов'] += 1
                    last['Дата Т'].append(t)
                    last['Сообщений Т'] += test['Сообщений Т']
    if test['Дата П'] == 'нет':
        pass
    else:
        if last['Дата П'] == 'нет':
                last['Игры ПКПК'] = test['Игры ПКПК']
                last['Роль П'] = test['Роль П']
                last['Дата П'] = test['Дата П']
                last['Баланс П'] = test['Баланс П']
        else:
            for i, t in enumerate(test['Дата П']):
                if t not in last['Дата П']:
                    last['Игры ПКПК'] += 1
                    last['Роль П'].append(test['Роль П'][i])
                    last['Дата П'].append(t)
                    last['Баланс П'].append(test['Баланс П'][i])
    return(last)

In [82]:
import PySimpleGUI as sg
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pandas.io.json import dumps
import pandas as pd
import json
import docx
import regex
import re
import os
import requests
import datetime as dt

time_start = dt.datetime.now()

auth()

print(dt.datetime.now() - time_start)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=922428539074-3lcp6a62nd735nvdh38lj548n223snd0.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Файл скачан: Nice2MeetYou.xlsx.
Первичный Процент, всего 6922 строк
Файл скачан: 24.10.21 Нальчик.json.
Telegram: 1
Файл скачан: 24.10.21 Нальчик level_5.html.
PKPK: 1
0:00:14.583834


In [83]:
time_start = dt.datetime.now()

time = dt.datetime(2021, 10, 24)

get_names_complete(fileNice2, filePrimaryP, filesTelegram, filesPKPK, time)
dff = df.reset_index()
url = 'https://10education.ru/CompleteSearch/select.php'
df = select_df(url)

print(dt.datetime.now() - time_start)

9
17
48
0:00:04.253243


In [84]:
dff

,Имя,Город проживания,Онлайн-кабинет,Дата о-к,Первичный П,Telegram каналов,Дата Т,Сообщений Т,Игры ПКПК,Роль П,Дата П,Баланс П
0,Елизавета,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
1,Антон Шпет Сергеевич,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
2,Харитонова Валерия Евгеньевна,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
3,екатерина,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
4,Урусова Анастасия Викторовна,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
5,Игнатович Евгений Алексеевич,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
6,Саргсян Лаура Арменовна,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
7,Валерия Курочкина,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
8,Лочмелис Алиса Андреевна,Омск,1,24.10.2021,0,0,нет,0,0,нет,нет,нет
9,Иван,не указан,0,нет,0,1,[24.10.21 Нальчик],0,0,нет,нет,нет


In [85]:
time_start = dt.datetime.now()
from tqdm import tqdm

url = 'https://10education.ru/CompleteSearch/update.php'
for i in tqdm(range(len(dff))):
    test = dff.iloc[i].to_dict()
    if test['Имя'] in df['Имя'].values:
        last = df[df['Имя'] == test['Имя']].to_dict('records')[0]
        test = update_row(last, test)
    for key, value in test.items():
        if isinstance(value, list):
            test[key] = ", ".join(map(str, value))
    r = requests.post(url, data = dumps(test))
    
print(dt.datetime.now() - time_start)

100%|██████████████████████████████████████████| 48/48 [00:09<00:00,  4.82it/s]

0:00:09.961570
